In [5]:
import numpy as np
import cv2
import time

In [6]:
# Using MeanShift for face tracking (no window resizing)
cap = cv2.VideoCapture(0)
# This delay was added because the first frame when starting the built-in camera was pitch black
# So added delay so brigthness calibration could be executed before grabbing sampling frame
time.sleep(2)
ret, frame = cap.read()

# Performing face tracking (only 1 face)
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

(face_x,face_y,w,h) = tuple(face_rects[0])

track_window = (face_x,face_y,w,h) 
roi = frame[face_y : face_y + h, face_x : face_x + w]

# MeanShift operates based on color histograms, so we need hsv mapping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setting termination criteria 
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        #         Now that we have calculated the hist mean, we will apply the MeanShift algorithm
        ret, track_window = cv2.meanShift(dst,track_window,term_crit)

        #     Update the window that is looking for the convergence of the match and values
        x,y,w,h = track_window
        img = cv2.rectangle(frame,(x,y),(x + w, y + h), (0,0,255),5)
        cv2.imshow('Tracking your face...', img)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [17]:
# Using CAMShift for face tracking (window resizing)
import numpy as np
import cv2
import time

In [18]:
cap = cv2.VideoCapture(0)
time.sleep(2)
ret, frame = cap.read()

# Performing face tracking (only 1 face)
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h) 
roi = frame[face_y : face_y + h, face_x : face_x + w]

# MeanShift operates based on color histograms, so we need hsv mapping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setting termination criteria 
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        #         Now that we have calculated the hist mean, we will apply the CAMShift algorithm 
        ret, track_window = cv2.CamShift(dst,track_window,term_crit)

        #        Resizing window based on the returned object from CamShift
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        
        #         Drawing the window's new lines on the image
        img = cv2.polylines(frame,[pts],True,(255,255,255),5)
        
        cv2.imshow('Tracking your face...', img)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1